# notebook for generating lowTh events

This is a roundabout solution that *will be changed*.

Start with a bunch of the fun IC packages that you want (fit this to your flavour of IC)

In [2]:
import sys,os,os.path
sys.path.append("../../../") # if you move files around, you need to adjust this!
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs

import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.database.load_db       import DataSiPM

from IC.invisible_cities.evm.event_model        import Cluster, Hit
from IC.invisible_cities.types.ic_types         import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits

from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod

import IC.invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize



### Pull out a couple of functions that may be deprecated, i can't remember

But they're needed for lowTh creation, and I wanted them in this file to better understand what I'm doing.

In [8]:


def threshold_hits(threshold_charge, same_peak, hitc):
    """
    Applies a threshold to hits and redistributes the charge/energy.

    Parameters
    ----------
    threshold_charge : float
        minimum pes of a hit
    same_peak        : bool
        whether to reassign NN hits' energy only to the hits from the same peak

    Returns
    ----------
    A function that takes HitCollection as input and returns another object with
    only non NN hits of charge above threshold_charge.
    The energy of NN hits is redistributed among neighbors.
    """

    t = hitc.time
    thr_hits = hif.threshold_hits(hitc.hits, threshold_charge     )
    mrg_hits = hif.merge_NN_hits ( thr_hits, same_peak = same_peak)

    cor_hits = []
    for hit in mrg_hits:
        cluster = Cluster(hit.Q, xy(hit.X, hit.Y), hit.var, hit.nsipm)
        xypos   = xy(hit.Xpeak, hit.Ypeak)
        hit     = Hit(hit.npeak, cluster, hit.Z, hit.E, xypos, hit.Ec)
        cor_hits.append(hit)

    new_hitc      = HitCollection(hitc.event, t)
    new_hitc.hits = cor_hits
    return new_hitc





def hits_from_df (dst, skip_NN = False):
    """
    Function that transforms pandas DataFrame dst to HitCollection
    ------
    Parameters
    ------
    dst : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
        If time, nsipm, Xrms, Yrms, Qc, Ec, track_id are not
        inside dst the default value is set to -1
        If Xpeak, Ypeak not in dst the default value is -1000
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    https://github.com/next-exp/IC/blob/v2-development/invisible_cities/io/hits_io.py#L16
    """
    all_events = {}
    times = getattr(dst, 'time', [-1]*len(dst))
    for (event, time) , df in dst.groupby(['event', times]):
        #pandas is not consistent with numpy dtypes so we have to change it by hand
        event = np.int32(event)
        hits  = []
        for i, row in df.iterrows():
            Q = getattr(row,'Q', row.E)
            if skip_NN and Q == NN:
                continue
            if hasattr(row, 'Xrms'):
                Xrms  = row.Xrms
                Xrms2 = Xrms**2
            else:
                Xrms = Xrms2 = -1
            if hasattr(row, 'Yrms'):
                Yrms  = row.Yrms
                Yrms2 = Yrms**2
            else:
                Yrms = Yrms2 = -1
            nsipm   = getattr(row, 'nsipm'   , -1   )     # for backwards compatibility
            Qc      = getattr(row, 'Qc'      , -1   )     # for backwards compatibility
            Xpeak   = getattr(row, 'Xpeak'   , -1000)     # for backwards compatibility
            Ypeak   = getattr(row, 'Ypeak'   , -1000)     # for backwards compatibility
            Ec      = getattr(row, 'Ec'      , -1   )     # for backwards compatibility
            trackID = getattr(row, 'track_id', -1   )     # for backwards compatibility
            Ep      = getattr(row, "Ep"      , -1   )     # for backwards compatibility

            hit = Hit(row.npeak            ,
                      Cluster(Q               ,
                              xy(row.X, row.Y),
                              xy(Xrms2, Yrms2),
                              nsipm = nsipm   ,
                              z     = row.Z   ,
                              E     = row.E   ,
                              Qc    = Qc      ),
                      row.Z                ,
                      row.E                ,
                      xy(Xpeak, Ypeak)     ,
                      s2_energy_c = Ec     ,
                      track_id    = trackID,
                      Ep          = Ep     )

            hits.append(hit)

        if len(hits):
            all_events[event] = HitCollection(event, time, hits=hits)

    return all_events

def hitc_to_df_(hitc):
    columns = defaultdict(list)
    for hit in hitc.hits:
        columns["event"   ].append(hitc.event)
        columns["time"    ].append(hitc.time)
        columns["npeak"   ].append(hit .npeak)
        columns["Xpeak"   ].append(hit .Xpeak)
        columns["Ypeak"   ].append(hit .Ypeak)
        columns["nsipm"   ].append(hit .nsipm)
        columns["X"       ].append(hit .X)
        columns["Y"       ].append(hit .Y)
        columns["Xrms"    ].append(hit .Xrms)
        columns["Yrms"    ].append(hit .Yrms)
        columns["Z"       ].append(hit .Z)
        columns["Q"       ].append(hit .Q)
        columns["E"       ].append(hit .E)
        columns["Qc"      ].append(hit .Qc)
        columns["Ec"      ].append(hit .Ec)
        columns["track_id"].append(hit .track_id)
        columns["Ep"      ].append(hit .Ep)
    return pd.DataFrame(columns)


def soph_to_lowTh(df, threshold = 5):
    '''
    Converts sophronia 'RECO/Events' to lowTh events via a rather convoluted process

    ------
    Parameters
    ------
    df : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
    threshold: int
        value at which the threshold is set.
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    '''

    # safety check, to ensure you don't accidentally make a repeating dataframe
    


    # new parameters for threshold, this is silly but I'm copying previous convention
    pes = 1
    threshold = threshold * pes
    same_peak = True

    # convert sophronia RECO/Events to hit collection
    soph_hitc = hits_from_df(df)

    # collect the keys as the event numbers
    soph_hitc_list = list(soph_hitc.keys())

    print("Processing data...")
    # loop over all of these events
    j = 0
    for i in soph_hitc_list:
        j += 1

        if (len(soph_hitc_list)%j == 50): 
            print("{}/{}".format(j, len(soph_hitc_list)))
        # choose i'th event
        soph_hit_event = soph_hitc.get(i)

        # Apply threshold calculation
        soph_hitc_lowTh = threshold_hits(threshold, same_peak, soph_hit_event)

        # convert back to pandas dataframe using hitc_to_df
        soph_hdst_lowTh = hitc_to_df_(soph_hitc_lowTh)

        # check if pandas dataframe with all the events exists yet
        if 'full_soph_df' in locals() and isinstance(full_soph_df, pd.DataFrame):
            full_soph_df = pd.concat([full_soph_df, soph_hdst_lowTh])
        else:
            full_soph_df = soph_hdst_lowTh.copy(deep = True)
    
    return full_soph_df

        


### EXAMPLE CASE: using 1 sophronia file

In [9]:
# loading sophronia file
soph_file = f'../../../Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/sophronia/sophronia_180_208Tl.h5'
soph_hdst = dstio.load_dst(soph_file, 'RECO', 'Events')#CHITS/lowTh
display(soph_hdst)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,3580000,3.580001e+12,0,-148.667262,364.111673,1,-272.125,147.725,0.0,0.0,374.378021,7.246377,765.075420,-1.0,0.002455,-1,-1.0
1,3580000,3.580001e+12,0,-148.667262,364.111673,1,-272.125,163.275,0.0,0.0,374.378021,5.043478,532.492472,-1.0,0.001720,-1,-1.0
2,3580000,3.580001e+12,0,-148.667262,364.111673,1,-256.575,147.725,0.0,0.0,374.378021,5.739130,605.939696,-1.0,0.001951,-1,-1.0
3,3580000,3.580001e+12,0,-148.667262,364.111673,1,-256.575,163.275,0.0,0.0,374.378021,5.043478,532.492472,-1.0,0.001721,-1,-1.0
4,3580000,3.580001e+12,0,-148.667262,364.111673,1,-272.125,147.725,0.0,0.0,375.409210,13.159420,597.688660,-1.0,0.001918,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175387,3580710,3.580712e+12,0,373.853623,76.970409,1,194.375,132.175,0.0,0.0,695.576599,8.869565,627.508728,-1.0,0.002034,-1,-1.0
175388,3580710,3.580712e+12,0,373.853623,76.970409,1,194.375,147.725,0.0,0.0,695.576599,6.260870,442.947357,-1.0,0.001432,-1,-1.0
175389,3580710,3.580712e+12,0,373.853623,76.970409,1,163.275,132.175,0.0,0.0,696.634155,7.072464,723.569092,-1.0,0.002337,-1,-1.0
175390,3580710,3.580712e+12,0,373.853623,76.970409,1,194.375,132.175,0.0,0.0,696.634155,7.188406,735.430908,-1.0,0.002384,-1,-1.0


In [10]:
# convert to hitcollection
soph_hits = hits_from_df(soph_hdst)

## Demonstrating for 1 event:

In [18]:
# set the event that you want 
event = 1

soph_hits_list = list(soph_hits.keys())
event_soph_hits = soph_hits_list[event]

print(event_soph_hits)

soph_hit_event = soph_hits.get(event_soph_hits)

#print(soph_hit_event)

3580002


### Here is where you define the threshold

lowTh being 5 * pes here

In [19]:
# new parameters for threshold
pes = 1
threshold = 5 * pes
same_peak = True

# Apply threshold calculation
soph_hitc_lowTh = threshold_hits(threshold, same_peak, soph_hit_event)

### convert back to pandas dataframe using hitc_to_df

In [20]:
# convert back to pandas dataframe using hitc_to_df
soph_hdst_lowTh = hitc_to_df_(soph_hitc_lowTh)

In [21]:
display(soph_hdst_lowTh)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,3580002,3.580002e+12,0.0,290.019426,197.375029,1.0,225.475,209.925,0.0,0.0,333.402771,6.144928,1945.000000,-1,0.006219,-1,-1
1,3580002,3.580002e+12,0.0,290.019426,197.375029,1.0,225.475,194.375,0.0,0.0,334.420959,6.086957,548.071756,-1,0.001745,-1,-1
2,3580002,3.580002e+12,0.0,290.019426,197.375029,1.0,225.475,209.925,0.0,0.0,334.420959,6.840580,615.928244,-1,0.001969,-1,-1
3,3580002,3.580002e+12,0.0,290.019426,197.375029,1.0,225.475,194.375,0.0,0.0,335.439026,6.840580,1123.000000,-1,0.003576,-1,-1
4,3580002,3.580002e+12,0.0,290.019426,197.375029,1.0,225.475,194.375,0.0,0.0,336.499573,5.101449,1832.000073,-1,0.005834,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,3580002,3.580002e+12,2.0,185.283977,160.488775,1.0,178.825,147.725,0.0,0.0,446.749573,6.840580,319.223579,-1,0.001002,-1,-1
759,3580002,3.580002e+12,2.0,185.283977,160.488775,1.0,178.825,178.825,0.0,0.0,446.749573,6.318841,294.876023,-1,0.000927,-1,-1
760,3580002,3.580002e+12,2.0,185.283977,160.488775,1.0,194.375,163.275,0.0,0.0,446.749573,10.318841,481.540685,-1,0.001515,-1,-1
761,3580002,3.580002e+12,2.0,185.283977,160.488775,1.0,209.925,163.275,0.0,0.0,446.749573,5.043478,235.359775,-1,0.000745,-1,-1


### This can be done for a full dataframe rather than one event

With this fun function I made :)

Be careful it takes a while.

In [22]:
soph_lowTh = soph_to_lowTh(soph_hdst, threshold = 5)
display(soph_lowTh)

Processing data...
60/350
75/350
100/350
150/350
